# Task 1: JSMA Attack

## Dependencies

In [ ]:
import pickle
import tensorflow as tf
import os
from torch import optim
import numpy as np

### If you are using Google Colab, you need to upload this notebook and the codebase to your Google Drive. Then you need to mount your Google Drive in Colab and set your working directory. If you are running on your local machine, you can ignore the following line.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
root_dir = "/content/drive/My Drive/"
project_dir = "Assignment1_code" # Change to your path
os.chdir(root_dir + project_dir)

In [ ]:
# Make sure the path is correct
!ls

CS5562_Assignment_1_Task_1.ipynb    imagenet_class_index.json
CS5562_Assignment_1_Task_2.ipynb    JSMA
CS5562_Assignment_1_Task_3.ipynb    model.py
CS5562_Assignment_1_Task_4.ipynb    __pycache__
CS5562_Assignment_1_Task_5.ipynb    results
CS5562_Assignment_1_Warm_ups.ipynb  test_image
defense.py			    utilities.py
environment.yml


### Importing helper functions

In [ ]:
from JSMA.pixelmap import AlgorithmEnum, Homography
from JSMA.utilities import generate_hull_mask, plot_and_save_graph, SDC_data
from utilities import *

In [ ]:
def JSMA_image(result_dir, attack_dir, is_mask=True, pixelmap=None, max_iters=50, debug_flag=False):
    # Credit to Jonathan Cheng for sharing the code.
    MODELS_DIR = 'JSMA/models'
    MODEL_NAME = 'sdc-50epochs-shuffled'
    IMAGE_FILE = None
    IMAGE_FOLDER = None
    RESULTS_DIR = None
    MAX_ITERATIONS = max_iters

    MODEL_PATH = os.path.join(MODELS_DIR, MODEL_NAME)

    # Path to attack folder
    IMAGE_FILE = os.path.join(attack_dir, 'attack.csv')
    IMAGE_FOLDER = attack_dir
    print('Using attack target images from: ' + IMAGE_FOLDER)
    print('Using attack target labels from: ' + IMAGE_FILE)

    RESULTS_DIR = os.path.join('results/', result_dir)
    if not os.path.isdir(RESULTS_DIR):
        os.makedirs(RESULTS_DIR)

    if pixelmap is not None:
        # Set the enum for which algorithm to use
        if pixelmap not in ['homography']:
            raise ValueError('pixelmap is one of: "homography"')
        if pixelmap == 'homography':
            PIXELMAP_ALGO = AlgorithmEnum.HOMOGRAPHY
    else:
        PIXELMAP_ALGO = None

    data = SDC_data(IMAGE_FILE, IMAGE_FOLDER)
    model = tf.keras.models.load_model(MODEL_PATH)

    attack = JSMARegressionAttack(model,
                                  RESULTS_DIR,
                                  is_mask=True,
                                  pixelmap_algo=PIXELMAP_ALGO,
                                  max_iters=MAX_ITERATIONS,
                                  )

    input_imgs = data.input_data
    adv_imgs = attack.attack(data)

## Complete the attack algorithm by filling in the TODO blocks

In [ ]:
class JSMARegressionAttack:
    """
    The JSMA attack. Credit to Jonathan Cheng for sharing the code.
    Returns adversarial examples for the supplied model.
    model: The model on which we perform the attack on.
    max_iters: The maximum number of iterations.
      Corresponds to the number of (pixel, colour) coordinates to perturb
    pixelmap_algo: Which mapping algorithm to use for the pixel mapping (don't map if None)
    clip_max: Maximum pixel value (default 1.0).
    clip_min: Minimum pixel value (default 0.0).
    increase: Direction of pixel values to perturb towards
      Does NOT affect the adversarial steering angles
    is_mask: Flag; Do we constraint the pertubation to a speciific region?
    """

    def __init__(self, model, resultsdir, max_iters=50, pixelmap_algo=None, clip_max=1.0, clip_min=0.0,
                 increase=True, is_mask=True):
        self.model = model
        self.resultsdir = resultsdir
        self.max_iters = max_iters
        self.pixelmap_algo = pixelmap_algo
        self.clip_max = clip_max
        self.clip_min = clip_min
        self.increase = increase
        self.is_mask = is_mask
        self.adv_preds = []
        self.adv_diffs = []

    def diff_avg(self, adv_diffs):
        return sum(adv_diffs) / len(adv_diffs)

    def seq_diff_avg(self, adv_diffs):
        adv_diffs_copy = adv_diffs.copy()
        seq_diffs = list(map(lambda pair: abs(pair[0] - pair[1]),
                             zip(adv_diffs_copy[1:], adv_diffs_copy[:len(adv_diffs_copy) - 1])))
        return sum(seq_diffs) / len(seq_diffs)

    def visualize_attack(self, batch_size, deltas, imgs):
        adv_preds = tf.squeeze(tf.stack(self.adv_preds, axis=1)).numpy()
        with open(os.path.join(self.resultsdir, 'preds.pkl'), 'wb') as f:
            pickle.dump(adv_preds.tolist(), f)
        plot_and_save_graph(adv_preds,
                            title='Predictions_Over_Rounds',
                            xlabel='Rounds',
                            ylabel='Adversarial Predictions',
                            savedir=self.resultsdir)
        adv_diffs = tf.squeeze(tf.stack(self.adv_diffs, axis=1)).numpy()
        with open(os.path.join(self.resultsdir, 'adv_diffs.pkl'), 'wb') as f:
            pickle.dump(adv_diffs.tolist(), f)
        plot_and_save_graph(adv_diffs,
                            title='Predictions_Difference_Over_Rounds',
                            xlabel='Rounds',
                            ylabel='Adversarial Predictions (Difference)',
                            savedir=self.resultsdir)
        adv_imgs = imgs + deltas
        # Save images of the deltas
        for i in range(batch_size):
            fig = plt.figure(frameon=False)
            ax = plt.Axes(fig, [0., 0., 1., 1.])
            ax.set_axis_off()
            fig.add_axes(ax)
            ax.imshow(tf.cast(deltas[i] != 0.0, tf.float32), interpolation='none')
            fig.savefig(os.path.join(self.resultsdir, 'delta' + str(i) + '.png'), dpi=250)
            plt.close(fig)
        # Save images of the perturbed target
        for i in range(batch_size):
            fig = plt.figure(frameon=False)
            ax = plt.Axes(fig, [0., 0., 1., 1.])
            ax.set_axis_off()
            fig.add_axes(ax)
            ax.imshow(adv_imgs[i], interpolation='none')
            fig.savefig(os.path.join(self.resultsdir, 'adv_image' + str(i) + '.png'), dpi=250)
            plt.close(fig)
        final_abs_adv_diffs = list(map(lambda diff: abs(diff), tf.squeeze(adv_diffs[:, -1:]).numpy().tolist()))
        print('Average difference:              ', self.diff_avg(final_abs_adv_diffs))
        print('Average sequantial difference:   ', self.seq_diff_avg(final_abs_adv_diffs))
        score = open(os.path.join(self.resultsdir, "score"), "w")
        score.write('Average difference:              ' + str(self.diff_avg(final_abs_adv_diffs)) + '\n')
        score.write('Average sequantial difference:   ' + str(self.seq_diff_avg(final_abs_adv_diffs)) + '\n')
        score.close()
        return adv_imgs

    def attack(self, data):
        # Sanity checks on data
        imgs, targets = data.input_data, data.output_data
        assert (len(imgs) == len(targets))
        if self.is_mask:
            sequence_of_list_of_corners = data.sequence_of_list_of_corners
            assert (len(sequence_of_list_of_corners) == len(imgs))

        print('Number of attack targets:    ', len(imgs))
        if self.is_mask:
            if self.pixelmap_algo:
                deltas = self.attack_pixelmap(data)
            else:
                deltas = self.attack_batch(data)
            adv_imgs = self.visualize_attack(len(imgs), deltas, imgs)
            return adv_imgs
        else:
            raise NotImplementedError('Not required for this assignment')

    def attack_batch(self, data):
        """
        Run the attack on a batch of images and labels.
        """
        imgs = data.input_data
        labs = data.output_data
        batch_size = len(imgs)
        sequence_of_list_of_corners = data.sequence_of_list_of_corners

        x = tf.cast(tf.constant(imgs), tf.float32)
        y_true = tf.expand_dims(tf.cast(tf.constant(labs), tf.float32), axis=1)

        # Compute our initial search domain.  We optimize the initial search domain
        # by removing all features that are already at their maximum values (if
        # increasing input features---otherwise, at their minimum value).
        if self.increase:
            search_domains = tf.Variable(tf.reshape(tf.equal(tf.cast(x < self.clip_max, tf.float32), 1.0), x.shape))
        else:
            search_domains = tf.Variable(tf.reshape(tf.equal(tf.cast(x > self.clip_min, tf.float32), 1.0), x.shape))

        # Manually calculate the allowed perturbation area
        # search_domains: A boolean mask to apply over the input images
        # search_domain_indices: A list (tensor) of vertices of the input images that are allowed to be perturbed
        if self.is_mask:
            for i in range(batch_size):
                list_of_corners = sequence_of_list_of_corners[i]
                pertubation_mask = generate_hull_mask(x.shape[1:3], np.array(list_of_corners))
                search_domains = search_domains[i].assign(
                    tf.math.logical_and(search_domains[i], tf.expand_dims(pertubation_mask, axis=2)))
        sparse_search_domains = tf.sparse.from_dense(tf.cast(tf.where(search_domains, x=1., y=0.), dtype=tf.float32))
        search_domain_indices = sparse_search_domains.indices

        # Create the variable tensor to calculate the jacobian
        deltas = tf.Variable(initial_value=tf.zeros_like(x),
                             shape=x.shape,
                             name='deltas',
                             dtype=tf.float32)

        original_pred = self.model.predict(x)

        # List of adversarial predictions and difference against original predictions in each iteration
        self.adv_preds.append(original_pred)
        self.adv_diffs.append(tf.zeros(shape=[batch_size, 1]))

        for i in range(self.max_iters):
            print('Rounds of perturbation:  ', i + 1)

            # Construct the computation graph to calculate the gradients (Jacobians)
            with tf.GradientTape(persistent=False, watch_accessed_variables=False) as tape:
                tape.watch(deltas)
                y = self.model(x + deltas)
                mseloss = (y - y_true) * (y - y_true) * 1 / 2

            # Will have shape: (batch size, <image.shape>)
            jacs = tf.squeeze(tape.batch_jacobian(mseloss, deltas), axis=1)

            to_add = tf.Variable(tf.zeros_like(deltas, dtype=tf.float32))

            # TODO
            # 1) Use `search_domains` and/or `search_domain_indices` to find the next pixel to update
            # 2) Assign `to_add`, and keep track of pixels updated so you don't pick it again in the next iteration

            # Loop through every image
            for i in range(batch_size):
              # Get available pixels and their jacobians to perturb
              search_indices = tf.sparse.from_dense(tf.cast(tf.where(search_domains[i], x=1., y=0.), dtype=tf.float32)).indices
              allowed_jacs = tf.boolean_mask(jacs[i], search_domains[i])

              # Get pixel with highest jacobian in magnitude since changing that pixel will cause max harm .
              # High jacobian means high sensitivity to input pixel values
              highest_jac_ind = tf.math.argmax(tf.math.abs(allowed_jacs))
              a, b, c = tf.gather(search_indices, highest_jac_ind)

              # Set value of pixel to mag(1)
              # Assigning highest perturbation budget for maximum damage in each direction
              if allowed_jacs[highest_jac_ind] < 0:
                to_add[i, a, b, c].assign(-1)
              else:
                to_add[i, a, b, c].assign(1)

              # Remove pixel from available pixel set
              search_domains[i, a, b].assign(False)

            # End of TODO

            # Update deltas
            deltas.assign(deltas + to_add)
            adv_pred = self.model.predict(x + deltas)
            # Record the effectiveness of perturbation at this iteration
            self.adv_preds.append(adv_pred)
            self.adv_diffs.append(original_pred - adv_pred)
        return deltas

    def attack_pixelmap(self, data):
        """
        Run the attack on a batch of images and labels.
        Do so by pertubring pixels in parallel, mapping pixels using some specified algorithm
        """
        imgs = data.input_data
        labs = data.output_data
        batch_size = len(imgs)
        sequence_of_list_of_corners = data.sequence_of_list_of_corners

        x = tf.cast(tf.constant(imgs), tf.float32)
        y_true = tf.expand_dims(tf.cast(tf.constant(labs), tf.float32), axis=1)

        if self.pixelmap_algo == AlgorithmEnum.HOMOGRAPHY:
            for list_of_corners in sequence_of_list_of_corners:
                assert (not list_of_corners is None)
            pixelmap = Homography(sequence_of_list_of_corners, x.shape[1:3])

        # Create the variable tensor to calculate the jacobian
        deltas = tf.Variable(initial_value=tf.zeros_like(x),
                             shape=x.shape,
                             name='deltas',
                             dtype=tf.float32)

        original_pred = self.model.predict(x)

        # List of adversarial predictions and difference against original predictions in each iteration
        self.adv_preds.append(original_pred)
        self.adv_diffs.append(tf.zeros(shape=[batch_size, 1]))

        for i in range(self.max_iters):
            # If there are no more vertice strings left to perturbed, end early
            if pixelmap.get_length_of_vertice_strings() == 0:
                break

            # Construct the computation graph to calculate the gradients (Jacobians)
            with tf.GradientTape(persistent=False, watch_accessed_variables=False) as tape:
                tape.watch(deltas)
                y = self.model(x + deltas)
                mseloss = (y - y_true) * (y - y_true) * 1 / 2

            # Will have shape: (batch size, <image.shape>)
            jacs = tf.squeeze(tape.batch_jacobian(mseloss, deltas), axis=1)

            to_add = tf.Variable(tf.zeros_like(deltas, dtype=tf.float32))

            list_of_vertice_strings = pixelmap.get_list_of_vertice_strings()

            # TODO
            # 1) Use `list_of_vertice_strings` to find the best string of vertices to update in parallel
            # 2) Assign `to_add`, and update `pixelmap` by calling `delete_vertice_string()` so you don't use the same vertice string twice

            # get 2100 * 20 jacobian magnitudes
            all_jacs = []
            for indices in list_of_vertice_strings:
              selected_jacs = tf.gather_nd(jacs, indices)
              all_jacs.append(selected_jacs)

            all_jacs = tf.convert_to_tensor(all_jacs)

            # get average jacobian of each frame
            average_jac = tf.reduce_mean(all_jacs, 0)

            # count how many frames in each vertice string have jacobian higher than frame-avg
            jacs_higher_than_avg_count = tf.reduce_sum(tf.cast(tf.greater(all_jacs, average_jac), tf.float32), axis=1)

            # get vertice string with highest count
            chosen_index = tf.argmax(jacs_higher_than_avg_count)
            chosen_vertice_string = list_of_vertice_strings[chosen_index]

            # update to_add for 20 frames
            for b, p1, p2, c in chosen_vertice_string:
              to_add[b, p1, p2, c].assign(1)

            # remove vertice string from available vertice strings
            pixelmap.delete_vertice_string(chosen_index)

            # End of TODO

            # Update deltas
            deltas = deltas.assign(deltas + to_add)

            adv_pred = self.model.predict(x + deltas)
            # Record the effectiveness of perturbation at this iteration
            self.adv_preds.append(adv_pred)
            self.adv_diffs.append(original_pred - adv_pred)

        return deltas


# Test your code

Below is two code snippets to test your algorithm runs. You should observe that your attack algorithm results in non-zero differences.

### Objective 1

In [ ]:
JSMA_image(result_dir="test1", attack_dir="JSMA/attack_targets/attack_left")

Using attack target images from: JSMA/attack_targets/attack_left
Using attack target labels from: JSMA/attack_targets/attack_left/attack.csv


Number of attack targets:     20
1/1 [==============================] - 0s 122ms/step
Rounds of perturbation:   1
1/1 [==============================] - 0s 21ms/step
Rounds of perturbation:   2
1/1 [==============================] - 0s 21ms/step
Rounds of perturbation:   3
1/1 [==============================] - 0s 20ms/step
Rounds of perturbation:   4
1/1 [==============================] - 0s 21ms/step
Rounds of perturbation:   5
1/1 [==============================] - 0s 21ms/step
Rounds of perturbation:   6
1/1 [==============================] - 0s 21ms/step
Rounds of perturbation:   7
1/1 [==============================] - 0s 29ms/step
Rounds of perturbation:   8
1/1 [==============================] - 0s 21ms/step
Rounds of perturbation:   9
1/1 [==============================] - 0s 25ms/step
Rounds of perturbation:   10
1/1 [==============================] - 0s 24ms/step
Rounds of perturbation:   11
1/1 [==============================] - 0s 22ms/step
Rounds of perturbation:   12
1/1

Average difference:               6.190826010704041
Average sequantial difference:    0.5791924627203691


### Objective 2

In [ ]:
# This test takes longer to run in Colab.
JSMA_image(result_dir="test2", attack_dir="JSMA/attack_targets/attack_left", pixelmap="homography")

Using attack target images from: JSMA/attack_targets/attack_left
Using attack target labels from: JSMA/attack_targets/attack_left/attack.csv


Number of attack targets:     20
1/1 [==============================] - 0s 33ms/step


Average difference:               2.383523201942444
Average sequantial difference:    0.27321183054070725
